In [2]:
import mediapipe as mp
import cv2
import numpy as np

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Saving the Video

In [4]:
import time
cap = cv2.VideoCapture(0)
time.sleep(5)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc('P','I','M','1')
videoWriter = cv2.VideoWriter('distance.avi',fourcc,fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()

    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
    except Exception as e:
        print(e)
        break
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

Capturing Landmarks and Saving it to CSV

In [5]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

In [6]:
landmarks = ['class']  # Class is Y - target value
for val in range(1,33+1):
    landmarks+=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [10]:
with open('distance.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
# Function for exporting the landmarks to train
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints,0,action)

        with open('distance.csv',mode='a',newline='') as f: # a stands for append
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [12]:
cap = cv2.VideoCapture('distance.avi')
# initiate the holistic model
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,image = cap.read()
        # Recolor the feed
        if image is None: break
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recoloring the image back for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        k = cv2.waitKey(1)
        if k==97: # a key 
            export_landmark(results,0) # 0 -> narrow 
        if k==115:# s key
            export_landmark(results,1) # 1 -> correct
        if k==100:# d key
            export_landmark(results,2) # 2 -> wide

        cv2.imshow('Raw Cam Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Training the labelled Data on a Classification Model

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv('distance.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,2.0,0.723751,-0.049298,-0.642914,0.999329,0.735183,-0.065842,-0.624667,0.998386,0.738107,...,0.338775,0.756306,0.816191,0.948369,0.062415,0.943334,0.401415,0.922194,0.125347,0.949879
1,2.0,0.647041,0.013189,-0.746258,0.999486,0.660023,-0.007457,-0.734798,0.998607,0.665897,...,0.191027,0.763103,0.825008,0.940904,-0.041937,0.958081,0.399222,0.918982,-0.012959,0.959239
2,2.0,0.642887,-0.014336,-0.643614,0.999578,0.656632,-0.016645,-0.640461,0.998849,0.661174,...,0.263745,0.753119,0.818989,0.932040,0.000965,0.965213,0.401244,0.912100,0.076625,0.960741
3,2.0,0.642500,-0.022484,-0.660777,0.999593,0.657063,-0.028539,-0.656290,0.998995,0.661525,...,0.316265,0.713201,0.808949,0.918164,0.091916,0.969207,0.398415,0.907529,0.114551,0.958177
4,2.0,0.627493,-0.026405,-0.620959,0.999459,0.640899,-0.031612,-0.611016,0.998541,0.647435,...,0.254588,0.739560,0.811597,0.928399,0.076115,0.973136,0.396832,0.914108,0.047798,0.964807


In [18]:
X = df.drop('class',axis=1) # features
Y = df['class'] # label

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=1234)

In [20]:
# Training and Pipelining the Model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [21]:
# Pipelines
pipelines = {
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [22]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,Y_train)
    fit_models[algo] = model 

In [23]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [24]:
fit_models['rf'].predict(X_test)

array([0., 0., 1., 2., 2., 2., 2., 0., 0., 1., 2., 2., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 2., 1., 0., 2., 1., 1., 0., 2., 1., 0., 2., 2.,
       2., 1., 2., 1., 2., 0., 1., 1., 0., 1., 0., 2., 1., 2., 0., 0., 0.,
       1., 2., 1., 2., 2., 2., 0., 1., 2., 0., 1., 0., 2., 2., 0., 0., 1.,
       2., 2., 0., 1., 1., 0., 2., 2., 1., 2., 1., 1., 0., 1., 2., 1., 1.,
       1., 2., 2., 0., 0., 0., 0., 2., 2., 2., 1., 2., 2., 2., 0., 0., 2.,
       1., 1., 2., 1., 0., 0., 0., 1., 1., 1., 1., 0., 2., 0., 0., 2., 1.,
       1., 2., 1., 2., 0., 0., 1., 1., 1., 1., 2., 0., 2., 1., 1., 1., 1.,
       1., 0., 1., 1., 2., 1., 1., 1., 1., 1., 0., 0., 2., 1., 1., 2., 0.,
       1., 1., 2., 0., 0., 2., 0., 1., 1., 0., 2., 1., 1., 2., 1., 2., 2.,
       1., 1., 1., 2.])

Evaluate and Serialize the Model


In [25]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
import pickle

In [26]:
for algo,model in fit_models.items():
    Y_pred = model.predict(X_test)
    print(algo,accuracy_score(Y_test.values,Y_pred),
          precision_score(Y_test.values,Y_pred,average='weighted'),
          recall_score(Y_test.values,Y_pred,average='weighted'))

rf 0.9770114942528736 0.9771711366538952 0.9770114942528736
gb 0.9770114942528736 0.9770223542801538 0.9770114942528736


In [27]:
Y_pred = fit_models['gb'].predict(X_test)
Y_pred

array([0., 0., 1., 2., 2., 2., 2., 0., 0., 1., 2., 2., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 2., 1., 0., 2., 1., 1., 0., 2., 1., 0., 2., 2.,
       2., 1., 2., 1., 2., 0., 1., 1., 0., 1., 0., 2., 1., 2., 0., 0., 0.,
       1., 2., 1., 2., 2., 2., 0., 1., 2., 0., 1., 0., 2., 0., 0., 0., 0.,
       2., 2., 0., 1., 1., 0., 2., 2., 1., 2., 1., 1., 0., 1., 2., 1., 1.,
       1., 2., 2., 0., 0., 0., 0., 2., 2., 2., 1., 2., 2., 2., 0., 0., 2.,
       1., 1., 2., 1., 0., 0., 0., 1., 1., 1., 1., 0., 2., 0., 0., 2., 1.,
       1., 2., 1., 2., 0., 0., 1., 1., 1., 1., 2., 0., 2., 1., 1., 1., 1.,
       1., 0., 1., 1., 2., 1., 1., 1., 1., 1., 0., 0., 2., 1., 1., 2., 0.,
       1., 1., 2., 0., 0., 2., 0., 1., 1., 0., 2., 1., 1., 2., 1., 2., 2.,
       1., 1., 1., 2.])

In [30]:
with open('distance.pkl','wb') as f:
    pickle.dump(fit_models['gb'],f)

Making Detections with the Model

In [31]:
with open('distance.pkl','rb') as f:
    model = pickle.load(f)

In [34]:
cap = cv2.VideoCapture(0)
current_Stage = ''
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recolor it back
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Draw Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        try:
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            bodylang_prob = model.predict_proba(X)[0]
            bodylang_class = model.predict(X)[0]
            
            if bodylang_class==0.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                current_Stage = "Narrow"
            elif bodylang_class==1.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                current_Stage = "Correct"
            elif bodylang_class==2.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                current_Stage = "Wide"

            # Get status box
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

            # Display Class
            cv2.putText(image,'Class',
                        (95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_Stage,
                            (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Probability
            cv2.putText(image,'Prob',
                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(bodylang_prob[np.argmax(bodylang_prob)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Deadlifts Distance',image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        except Exception as e:
            print(e)
            pass   
cap.release()
cv2.destroyAllWindows()

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute